In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandarallel
import tqdm
import altair as alt

from altair_saver import save

In [ ]:
!git rev-parse --show-toplevel


In [ ]:
from pathlib import Path

In [ ]:
file = './nogit_data/list_of_post_contents.csv'

In [ ]:
df = pd.read_csv(file)#, nrows=10_00)
df

In [ ]:
df.columns

In [ ]:
columns_page_info = ['time_downloaded', 'author', 'posted_date_readable',  'post_ordinal', 'thread_page_name', 'thread_page_num', 'thread_page_url', 'post_text']

columns_thread_info = ['src_category_name', 'thread_page_name', 'thread_page_num', 'thread_max_pages', 'thread_page_url']

columns_likes = ['num_likers', 'likers']
columns_quotes = ['num_quotes', 'quoted_post_ids', 'quoted_authors', 'quoted_contents']
columns_authors = ['author', 'author_title', 'author_num_posts', 'author_num_reviews', 'author_url', 'join_date_readable', 'join_date_data']

In [ ]:
posts = df[columns_page_info].dropna(subset=['time_downloaded'])

In [ ]:
from tqdm import tqdm
tqdm.pandas()
# - Usage: replace df.apply() with progress_apply()

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
# - Usage: replace df.apply() with parallel_apply()

In [ ]:
posts['posted_date_datetime'] = posts.posted_date_readable.parallel_apply(lambda x: pd.to_datetime(x))

In [ ]:
posts.sample()

In [ ]:
#annot_text[(annot_text.index > '1990') & (annot_text.index < '2023')].resample('Y').count()

In [ ]:
#try: posts.set_index('posted_date_readable') 
#except: pass
#display(posts.dtypes)
#print(posts.index)
print(posts.dtypes)
#posts.groupby(posts.posted_date_datetime).count()
date_filter = (posts.posted_date_datetime > '2000') & (posts.posted_date_datetime < '2023')#.resample('Y').count().reset_index()
_tmp = posts[date_filter]
#_tmp.resample('Y', on='post_date_datetime').count().reset_index()
_tmp

# define weekly and daily

In [ ]:
# https://vega.github.io/vega/docs/schemes/

                                # https://stackoverflow.com/questions/60331614/how-to-highlight-a-bar-by-datetime-value-with-altair
# https://stackoverflow.com/questions/60351135/hours-and-minutes-as-labels-in-altair-plot-spanning-more-than-one-day
                                # https://stackoverflow.com/questions/60331614/how-to-highlight-a-bar-by-datetime-value-with-altair
#!pip install altair==5.0.0rc1
# must be at least 5.0.0 or else "method based syntax" does not work (for setting title)
# will get error "undefinedtype" object is not callable

#alt.themes.enable('default') # ggplot2, vox, quartz, fivethirtyeight, googlecharts, excel
# https://vega.github.io/vega-themes/?theme=default

        #ch = alt.Chart(_df).mark_line().encode(
        #x='posted_time:T',
        #y='posts_per_day:Q'

weekly = posts[date_filter].resample('W', on='posted_date_datetime').apply({'post_ordinal': 'count'}).reset_index(
).rename(columns={'posted_date_datetime': 'posted_time', 'post_ordinal': 'posts_per_day'})
weekly



In [ ]:
daily = posts[date_filter].resample('D', on='posted_date_datetime').apply({'post_ordinal': 'count'}).reset_index(
).rename(columns={'posted_date_datetime': 'posted_time', 'post_ordinal': 'posts_per_day'})
daily


# define theme

In [ ]:
# https://github.com/altair-viz/altair/issues/1021
#def finalize_chart(chart):
    #chart.x.axis = {'title': 'options'}
    #chart.y.axis = {'title': 'percentage of students'}
    #chart.mark = {'type': 'bar', 'size': 3)
    #return chart

alt.__version__ 
SCALE=1.5
def theme_1(*args, **kwargs):
    return {'width': 400, 'height': 300,
            'config': {#'style': {'bar': {'size': 20}},
                       #'legend': {'symbolSize': 20, 'titleFontSize': 20, 'labelFontSize': 20}, 
                       'title':{'fontSize': 20*SCALE},
                       'axis': {'titleFontSize': 15*SCALE, 'labelFontSize': 15*SCALE},
                        "axisX": { #https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602
                                "tickSize": 10, # default, including it just to show you can change it
                                "tickWidth":2,
                        }
            },
    }
            #'encoding': {'x': {'axis': {'title': 'options'}, 'scale': {'paddingOuter': 0.5, 'paddingInner': 0.5}},
            #             'y': {'axis': {'title': 'percentage of students'}}}}
alt.themes.register('theme_1', theme_1)
alt.themes.enable('theme_1')

# plot

# daily

In [ ]:
alt.themes.enable('theme_1')
#date_filter = alt.selection_interval
fmt_obj = '{ "year": "%Y", "quarter": "%B", "month": "%B", "week": "%b %d", "date": "%a %d", "hours": "%H:%M", "minutes": "%H:%M", "seconds": ":%S", "milliseconds": ".%L" }'
time_fmt = f"timeFormat(datum.value, {fmt_obj})"

# https://github.com/altair-viz/altair/issues/187
date_min = daily.posted_time.min()
date_max = daily.posted_time.max() 
#my_values = list(pd.date_range(date_min, date_max, freq='W-MON').astype(int) / 10 ** 6)
my_values = list(pd.date_range(date_min, date_max, freq='M').astype(int))
#display(my_values)
time_fmt = "month(datum.value) == 'June' ? '2019!' : timeFormat(datum.value)"

#.transform_calculate(
#    group='month(datum.date) < 4 ? "Jan-Apr" : month(datum.date) < 8 ? "May-Aug" : "Sep-Dec"'
chart = alt.Chart(daily).mark_bar(
    width=2,
    opacity=1,
    clip=True,
    #line=True,
    #stroke='gray',
    #strokeOpacity=8
    color='lightblue'
    ).encode(
        alt.X('posted_time:T',
                scale=alt.Scale(nice={'interval': 'month', 'step': 1}, domain=[daily.posted_time.min(), daily.posted_time.max()]),
                axis=alt.Axis(
                    #values=my_values,
                    #format={'month':'%B','year':'%Y'},
                    #format={'date':'%x', 'month':'%b \'%y', 'year':'%Y', 'quarter':'%b \'%y'},
                    #format={'date':'%x', 'month':'%b.', 'year':'%Y', 'quarter':'%b.'},
                    format={'date':'%x', 'month':'%b.', 'year':'{%Y}', 'quarter':'%b.'},
                    tickCount={"interval": "month", "step":1},
                    labelAngle=-30,
                    #labelExpr=time_fmt/
                    )
                ).title('Time (Tick per Month)'),
            alt.Y('posts_per_day:Q',).title('Num. of Posts'),
            #color=alt.condition(genre_filter, alt.value(), alt.value("red")),
                color = alt.condition(
                                ((alt.datum.posted_time > alt.expr.toDate('2020-03-01T00:00:00')) &
                                        (alt.datum.posted_time <= alt.expr.toDate('2020-07-30T00:00:00'))) |
                                (alt.datum.posted_time < alt.expr.toDate(
                                    '2018-12-31T00:00:00')),
                                alt.value('lightblue'), alt.value('lightblue')
                                ),
        ).properties(width=1200, height=250, title='Num. of Forum Posts (per day)')
#chart.configure_bar(stroke='gray')

covid_start = daily[daily.posted_time >= '2020-03-01'].iloc[0].values[0]
covid_end = daily[daily.posted_time >= '2020-07-31'].iloc[0].values[0]
cutoff = pd.DataFrame({
    'start': [covid_start],
    'stop': [covid_end ]
})

area_covid = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.5,
    stroke='red',
    strokeOpacity=1,
    strokeWidth=3
).encode(
    x='start',
    x2='stop',
    y=alt.value(0),  # pixels from top
    y2=alt.value(250),  # pixels from top
    color=alt.value('transparent'),
 
)
cutoff_2019 = pd.DataFrame({
    'start': [daily.posted_time.min()],
    'stop': [daily.posted_time.min()],
    #'stop': [daily[daily.posted_time >= '2018-12-31'].iloc[0].values[0]],
})
area_2019= alt.Chart(
    cutoff_2019.reset_index()
).mark_rect(
    opacity=0.5,
    stroke='red',
    strokeOpacity=1,
    strokeWidth=3
).encode(
    x='start',
    x2='stop',
    y=alt.value(0),  # pixels from top
    y2=alt.value(250),  # pixels from top
    color=alt.value('transparent'),
 
)

years = []
for year in range(2019,2023):
    year_dt = pd.to_datetime(str(year)) # CAREFUL! must use str() here!
    print(year_dt)
    years.append(daily[daily.posted_time >= year_dt].iloc[0].values[0])
rules = alt.Chart(pd.DataFrame({
  'Date': years, 
  #'color': 'orange'
})).mark_rule().encode(
  x='Date:T',
  color=alt.value('gray'),
  #color=alt.Color('color:N', scale=None)
      strokeWidth=alt.value(1.5)
)
#!pip install 
#(demo + rules )
#(areas + chart)
#(areas +  chart + rules)

final = (chart+area_covid+rules).configure(background='#F9F3DC').configure_view(fill='white')
#save( final, str(Path(LATEX_FIG_FOLDER, 'daily_posts.pdf')))

#chart
#overlay = pd.DataFrame({'x': [covid_date]})
#vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
#alt.layer(chart, vline, data=daily)

#chart.mark_rule().encode(x=alt.datum(.toDate('2020-03-01T00:00:00')))
#chart.mark_rule().encode(x={'Timestamp': "2020-01-01"},)
( chart + area_covid + rules)#area_2019 + rules)
final

In [ ]:
# to output to pdf... need... npm ??? wtf
# https://github.com/altair-viz/altair_saver#installation

# weekly

In [ ]:
alt.themes.enable('theme_1')
#date_filter = alt.selection_interval
fmt_obj = '{ "year": "%Y", "quarter": "%B", "month": "%B", "week": "%b %d", "date": "%a %d", "hours": "%H:%M", "minutes": "%H:%M", "seconds": ":%S", "milliseconds": ".%L" }'
time_fmt = f"timeFormat(datum.value, {fmt_obj})"

# https://github.com/altair-viz/altair/issues/187
date_min = weekly.posted_time.min()
date_max = weekly.posted_time.max() 
#my_values = list(pd.date_range(date_min, date_max, freq='W-MON').astype(int) / 10 ** 6)
my_values = list(pd.date_range(date_min, date_max, freq='M').astype(int))
#display(my_values)
time_fmt = "month(datum.value) == 'June' ? '2019!' : timeFormat(datum.value)"

#.transform_calculate(
#    group='month(datum.date) < 4 ? "Jan-Apr" : month(datum.date) < 8 ? "May-Aug" : "Sep-Dec"'
chart = alt.Chart(weekly).mark_bar(
    width=4,
    opacity=1,
    #line=True
    #stroke='gray',
    #strokeOpacity=8
    color='lightblue'
    ).encode(
        alt.X('posted_time:T',
                scale=alt.Scale(nice={'interval': 'month', 'step': 1}),
                axis=alt.Axis(
                    #values=my_values,
                    #format={'month':'%B','year':'%Y'},
                    #format={'date':'%x', 'month':'%b \'%y', 'year':'%Y', 'quarter':'%b \'%y'},
                    #format={'date':'%x', 'month':'%b.', 'year':'%Y', 'quarter':'%b.'},
                    format={'date':'%x', 'month':'%b', 'year':'%Y', 'quarter':'%b.'},
                    tickCount={"interval": "month", "step":1},
                    labelAngle=-30,
                    #labelExpr=time_fmt/
                    )
                ).title('Time (Tick per Month)'),
            alt.Y('posts_per_day:Q',).title('Num. of Posts'),
            #color=alt.condition(genre_filter, alt.value(), alt.value("red")),
                color = alt.condition(
                                ((alt.datum.posted_time > alt.expr.toDate('2020-03-01T00:00:00')) &
                                        (alt.datum.posted_time <= alt.expr.toDate('2020-07-30T00:00:00'))) |
                                (alt.datum.posted_time < alt.expr.toDate(
                                    '2018-12-31T00:00:00')),
                                alt.value('gold'), alt.value('lightblue')
                                ),
        ).properties(width=1000, height=250, title='Num. of Forum Posts (per week)')
#chart.configure_bar(stroke='gray')

covid_start = weekly[weekly.posted_time >= '2020-03-01'].iloc[0].values[0]
covid_end = weekly[weekly.posted_time >= '2020-07-31'].iloc[0].values[0]
cutoff = pd.DataFrame({
    'start': [covid_start],
    'stop': [covid_end ]
})
areas = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.5,
    stroke='red',
    strokeOpacity=1,
    strokeWidth=3
).encode(
    x='start',
    x2='stop',
    y=alt.value(0),  # pixels from top
    y2=alt.value(250),  # pixels from top
    color=alt.value('transparent'),
 
)

#forum_start = weekly.posted_time.min()
#forum_end = weekly[weekly.posted_time >= '2019-01-01'].iloc[0].values[0]
years = []
for year in range(2019,2023):
    year_dt = pd.to_datetime(str(year)) # CAREFUL! must use str() here!
    print(year_dt)
    years.append(weekly[weekly.posted_time >= year_dt].iloc[0].values[0])
rules = alt.Chart(pd.DataFrame({
  'Date': years, 
  #'color': 'orange'
})).mark_rule().encode(
  x='Date:T',
  color=alt.value('gray'),
  #color=alt.Color('color:N', scale=None)
      strokeWidth=alt.value(1.5)
)
#(demo + rules )
(areas + chart)
(areas +  chart + rules)
( chart + areas+ rules)
#chart.save('char.pdf')
#chart
#overlay = pd.DataFrame({'x': [covid_date]})
#vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
#alt.layer(chart, vline, data=weekly)

#chart.mark_rule().encode(x=alt.datum(.toDate('2020-03-01T00:00:00')))
#chart.mark_rule().encode(x={'Timestamp': "2020-01-01"},)

In [ ]:
type(weekly[weekly.posted_time >= '2020-03-01'].iloc[0])
covid_date = weekly[weekly.posted_time >= '2020-03-01'].iloc[0].values[0]
covid_date

In [ ]:
pd.to_datetime('2020')

# nah, use the background instead

In [ ]:
alt.themes.enable('theme_1')

fmt_obj = '{ "year": "%Y", "quarter": "%B", "month": "%B", "week": "%b %d", "date": "%a %d", "hours": "%H:%M", "minutes": "%H:%M", "seconds": ":%S", "milliseconds": ".%L" }'
time_fmt = f"timeFormat(datum.value, {fmt_obj})"

#.transform_calculate(
#    group='month(datum.date) < 4 ? "Jan-Apr" : month(datum.date) < 8 ? "May-Aug" : "Sep-Dec"'
chart = alt.Chart(weekly).mark_bar(
            color='lightblue'
    ).encode(
        alt.X('posted_time:T',
                #timeUnit='month'
                scale=alt.Scale(nice={'interval': 'month', 'step': 1}),
                axis=alt.Axis(
                    format={'date':'%x', 'month':' ', 'year':'%Y', 'quarter':' '},
                    tickCount={"interval": "month", "step":1},
                    ),
                ).title('Time'),
            alt.Y('posts_per_day:Q',).title('Num. of Posts'),
            #color=alt.condition(genre_filter, alt.value(), alt.value("red")),
        ).properties(width=1500, height=200, title='Posts per Week')
chart.configure_bar(stroke='gray')


covid_start = weekly[weekly.posted_time >= '2020-03-01'].iloc[0].values[0]
covid_end = weekly[weekly.posted_time >= '2020-07-31'].iloc[0].values[0]
cutoff = pd.DataFrame({
    'start': [covid_start],
    'stop': [covid_end ]
})
areas = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.9
).encode(
    x='start',
    x2='stop',
    y=alt.value(0),  # pixels from top
    y2=alt.value(200),  # pixels from top
    color=alt.value('gold')
)

(areas + chart)


In [ ]:

# unused example
#text = chart.mark_text(align='left', dx=5, dy=-5).encode(
    #text=alt.condition(alt.datum.posted_time == alt.expr.toDate(
                                    #'2018-12-31T00:00:00'),
                                    #'2018', alt.value('')
                                    #)
#)
#chart + text

In [ ]:
chart = alt.Chart(daily).mark_bar(
    ).encode(
        alt.X('yearquartermonth(posted_time):T',
                axis=alt.Axis(
                    #values=my_values,
                    #format={'month':'%B','year':'%Y'},
                    #tickCount={"interval": "month", "step":1},
                    #labelExpr=time_fmt
                    )
                ).title('Time'),
            alt.Y('sum(posts_per_day):Q').title('Num. of Posts'),
            #color=alt.condition(genre_filter, alt.value(), alt.value("red")),
                color = alt.condition(
                                ((alt.datum.posted_time > alt.expr.toDate('2020-03-01T00:00:00')) &
                                        (alt.datum.posted_time <= alt.expr.toDate('2020-07-30T00:00:00'))) |
                                (alt.datum.posted_time < alt.expr.toDate(
                                    '2018-12-31T00:00:00')),
                                alt.value('gold'), alt.value('lightblue')
                                ),
        ).properties(width=1050, height=200, title='Posts per Week')

#x_axis = alt.X('time', axis=alt.Axis(labelExpr=time_fmt))

chart.configure_bar(stroke='gray')


In [ ]:
# https://vega.github.io/vega/docs/schemes/
import altair as alt
#!pip install altair==5.0.0rc1
display(alt.__version__ )# must be at least 5.0.0 or else "method based syntax" does not work (for setting title)     
# will get error "undefinedtype" object is not callable

alt.themes.enable('default') # ggplot2, vox, quartz, fivethirtyeight, googlecharts, excel
# https://vega.github.io/vega-themes/?theme=default

_df = _tmp.resample('D', on='posted_date_datetime').apply({'post_ordinal':'count'}).reset_index().rename(columns={'posted_date_datetime':'posted_time', 'post_ordinal':'posts_per_day'})
_df

chart = alt.Chart(_df).mark_bar(
    #line=True
).encode(
#ch = alt.Chart(_df).mark_line().encode(
    #x='posted_time:T',
    #y='posts_per_day:Q'
    alt.X('posted_time:T').title('Day'),
    alt.Y('posts_per_day:Q').title('Num. of Posts'),

).properties(width=1000, height=400, title='Posts per Day')

chart

In [ ]:
# temps = temps[temps.date < '2010-01-15']
# alt.Chart(temps).mark_line().encode(
#     x='date:T',
#     y='temp:Q'
# )
#alt.data_transformers.enable('data_server')
# https://altair-viz.github.io/user_guide/times_and_dates.html
#alt.Chart(posts.reset_index().melt("posted_date_datetime")).mark_circle().encode(
    #x='posted_date_datetime',
    #y='value',
#).properties(width=800, height=400)

In [ ]:
df.columns

In [ ]:
df.author_num_posts

In [ ]:
import altair as alt
print(alt.__version__)
from vega_datasets import data

temps = data.seattle_temps()
temps.head()
temps = temps[temps.date < '2010-01-15']
alt.Chart(temps).mark_line().encode(
    x='date:T',
    y='temp:Q'
)

In [ ]:
alt.themes.enable('theme_1')
#date_filter = alt.selection_interval
fmt_obj = '{ "year": "%Y", "quarter": "%B", "month": "%B", "week": "%b %d", "date": "%a %d", "hours": "%H:%M", "minutes": "%H:%M", "seconds": ":%S", "milliseconds": ".%L" }'
time_fmt = f"timeFormat(datum.value, {fmt_obj})"

# https://github.com/altair-viz/altair/issues/187
date_min = daily.posted_time.min()
date_max = daily.posted_time.max() 
#my_values = list(pd.date_range(date_min, date_max, freq='W-MON').astype(int) / 10 ** 6)
my_values = list(pd.date_range(date_min, date_max, freq='M').astype(int))
#display(my_values)
time_fmt = "month(datum.value) == 'June' ? '2019!' : timeFormat(datum.value)"

#.transform_calculate(
#    group='month(datum.date) < 4 ? "Jan-Apr" : month(datum.date) < 8 ? "May-Aug" : "Sep-Dec"'
chart = alt.Chart(daily).mark_bar(
    width=2,
    opacity=1,
    #line=True
    #stroke='gray',
    #strokeOpacity=8
    color='lightblue'
    ).encode(
        alt.X('posted_time:T',
                scale=alt.Scale(nice={'interval': 'month', 'step': 1}),
                axis=alt.Axis(
                    #values=my_values,
                    #format={'month':'%B','year':'%Y'},
                    #format={'date':'%x', 'month':'%b \'%y', 'year':'%Y', 'quarter':'%b \'%y'},
                    #format={'date':'%x', 'month':'%b.', 'year':'%Y', 'quarter':'%b.'},
                    format={'date':'%x', 'month':'%b', 'year':'%Y', 'quarter':'%b.'},
                    tickCount={"interval": "month", "step":1},
                    labelAngle=-30,
                    #labelExpr=time_fmt/
                    )
                ).title('Time (Tick per Month)'),
            alt.Y('posts_per_day:Q',).title('Num. of Posts'),
            #color=alt.condition(genre_filter, alt.value(), alt.value("red")),
                color = alt.condition(
                                ((alt.datum.posted_time > alt.expr.toDate('2020-03-01T00:00:00')) &
                                        (alt.datum.posted_time <= alt.expr.toDate('2020-07-30T00:00:00'))) |
                                (alt.datum.posted_time < alt.expr.toDate(
                                    '2018-12-31T00:00:00')),
                                alt.value('lightblue'), alt.value('lightblue')
                                ),
        ).properties(width=1200, height=250, title='Num. of Forum Posts (per day)')
#chart.configure_bar(stroke='gray')

covid_start = daily[daily.posted_time >= '2020-03-01'].iloc[0].values[0]
covid_end = daily[daily.posted_time >= '2020-07-31'].iloc[0].values[0]
cutoff = pd.DataFrame({
    'start': [covid_start],
    'stop': [covid_end ]
})

area_covid = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.5,
    stroke='red',
    strokeOpacity=1,
    strokeWidth=3
).encode(
    x='start',
    x2='stop',
    y=alt.value(0),  # pixels from top
    y2=alt.value(250),  # pixels from top
    color=alt.value('transparent'),
 
)
cutoff_2019 = pd.DataFrame({
    'start': [daily.posted_time.min()],
    'stop': [daily.posted_time.min()],
    #'stop': [daily[daily.posted_time >= '2018-12-31'].iloc[0].values[0]],
})
area_2019= alt.Chart(
    cutoff_2019.reset_index()
).mark_rect(
    opacity=0.5,
    stroke='red',
    strokeOpacity=1,
    strokeWidth=3
).encode(
    x='start',
    x2='stop',
    y=alt.value(0),  # pixels from top
    y2=alt.value(250),  # pixels from top
    color=alt.value('transparent'),
 
)

years = []
for year in range(2019,2023):
    year_dt = pd.to_datetime(str(year)) # CAREFUL! must use str() here!
    print(year_dt)
    years.append(daily[daily.posted_time >= year_dt].iloc[0].values[0])
rules = alt.Chart(pd.DataFrame({
  'Date': years, 
  #'color': 'orange'
})).mark_rule().encode(
  x='Date:T',
  color=alt.value('gray'),
  #color=alt.Color('color:N', scale=None)
      strokeWidth=alt.value(1.5)
)
#(demo + rules )
#(areas + chart)
#(areas +  chart + rules)
( chart + area_covid + rules)#area_2019 + rules)
from altair_saver import save
save( ( chart + area_covid + rules), 'chart.pdf')
#chart
#overlay = pd.DataFrame({'x': [covid_date]})
#vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
#alt.layer(chart, vline, data=daily)

#chart.mark_rule().encode(x=alt.datum(.toDate('2020-03-01T00:00:00')))
#chart.mark_rule().encode(x={'Timestamp': "2020-01-01"},)

In [ ]:
user_info = df[columns_authors].dropna(subset='author').drop_duplicates(subset='author')
print(user_info.author_num_posts.iloc[3]) # string
print(user_info.author_num_posts.iloc[2]) #  float
user_info['author_num_posts'] = user_info['author_num_posts'].apply(lambda x: int(str(x).replace(',', ''))) # if type(x)=='str' else -99))
#_tmp= df['author_num_posts'].apply(lambda x: type(x)) 
user_info.sort_values(by='author_num_posts')

In [ ]:
#df[columns_authors].dropna().drop_duplicates(subset='author').sort_values(by='author_num_posts', ascending=False)

In [ ]:
#_tmp = user_info.sort_values(by='author_num_posts')['author', 'join_date_readable', 'author_num_posts']
user_info[user_info.author == 'Maestro714']

In [ ]:
user_info.author_num_posts.plot() # sort by join date maybe?